In [1]:
#Part 7 - Federated Learning with FederatedDataset

import torch as th
import syft as sy

epochs = 50

sy.create_sandbox(globals(), verbose=False)


Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\ProgramData\Anaconda3\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'



Setting up Sandbox...
Done!


In [2]:
boston_data = grid.search("#boston", "#data")
boston_target = grid.search("#boston", "#target")

In [3]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1

model = th.nn.Linear(n_features, n_targets)

In [4]:
# Cast the result in BaseDatasets
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Build the FederatedDataset object
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)
optimizers = {}
for worker in dataset.workers:
    optimizers[worker] = th.optim.Adam(params=model.parameters(),lr=1e-2)

['bob', 'theo', 'jason', 'alice', 'andy', 'jon']


In [5]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=32, shuffle=False, drop_last=False)

In [7]:
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        model.send(data.location)
        
        optimizer = optimizers[data.location.id]
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred.view(-1) - target)**2).mean()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 8 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBatch loss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))            
            
    print('Total loss', loss_accum)

Train Epoch: 1 [0/16 (0%)]	Batch loss: 9823.008789
Train Epoch: 1 [8/16 (50%)]	Batch loss: 2321.514648
Train Epoch: 1 [16/16 (100%)]	Batch loss: 47.774078
Total loss 63394.198066711426
Train Epoch: 2 [0/16 (0%)]	Batch loss: 120.189125
Train Epoch: 2 [8/16 (50%)]	Batch loss: 1038.533447
Train Epoch: 2 [16/16 (100%)]	Batch loss: 1471.783203
Total loss 26244.85154724121
Train Epoch: 3 [0/16 (0%)]	Batch loss: 319.875244
Train Epoch: 3 [8/16 (50%)]	Batch loss: 371.299316
Train Epoch: 3 [16/16 (100%)]	Batch loss: 61.601097
Total loss 7772.840141296387
Train Epoch: 4 [0/16 (0%)]	Batch loss: 205.691330
Train Epoch: 4 [8/16 (50%)]	Batch loss: 200.993286
Train Epoch: 4 [16/16 (100%)]	Batch loss: 804.882507
Total loss 5748.429527282715
Train Epoch: 5 [0/16 (0%)]	Batch loss: 372.154938
Train Epoch: 5 [8/16 (50%)]	Batch loss: 202.719574
Train Epoch: 5 [16/16 (100%)]	Batch loss: 164.997635
Total loss 4867.413619995117
Train Epoch: 6 [0/16 (0%)]	Batch loss: 82.889633
Train Epoch: 6 [8/16 (50%)]	Batch